In [ ]:
### imports

# external modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import importlib

# internal modules
import datagen
importlib.reload(datagen)
import count_objects
importlib.reload(count_objects)

In [ ]:
### generate an image and count the objects in it

# generate image
image_size = (256,256)
objects = {'blob': {1:10, 2:5, 10:3},
           'line': {15:4}}
im = datagen.generate_image( image_size=image_size, objects=objects )

# count objects
object_coords = count_objects.count_objects_cluster(im)

# compare counts to true values
ntrue = sum([sum(el.values()) for el in objects.values()])
nreco = len(object_coords)
print('True number of objects: {}'.format(ntrue))
print('Reconstructed number of objects: {}'.format(nreco))

# make a plot
fig,ax = plt.subplots( figsize=(12,12) )
ax.imshow( im, cmap='gray' )
for coord in object_coords:
    boxhalfwidth = 5
    patchxcoord = coord[1]
    patchycoord = coord[0]
    anchor = (patchxcoord-boxhalfwidth,patchycoord-boxhalfwidth)
    box = mpl.patches.Rectangle(anchor, 2*boxhalfwidth, 2*boxhalfwidth,
                               edgecolor='r', facecolor=(0,0,0,0))
    ax.add_patch(box)

In [ ]:
### generate many images and compare the reconstruction to the true values

nimages = 1500
nmaxobjects = 50
ntrue_hist = np.zeros(nmaxobjects+1)
nreco_hist = np.zeros(nmaxobjects+1)
rng = np.random.default_rng()
doplot = False

for i in range(nimages):

    # generate image
    image_size = (256,256)
    objects = {'blob': {1:int(rng.uniform()*15), 2:int(rng.uniform()*10), 10:int(rng.uniform()*5)},
           'line': {15:int(rng.uniform()*4)}}
    im = datagen.generate_image( image_size=image_size, objects=objects )

    # count objects
    object_coords = count_objects.count_objects_cluster(im)

    # compare counts to true values
    ntrue = sum([sum(el.values()) for el in objects.values()])
    nreco = len(object_coords)
    truebinn = min(nmaxobjects,ntrue)
    recobinn = min(nmaxobjects,nreco)
    ntrue_hist[truebinn] += 1
    nreco_hist[recobinn] += 1
    
    if( ntrue!=nreco and doplot ):
        
        print('True number of objects: {}'.format(ntrue))
        print('Reconstructed number of objects: {}'.format(nreco))

        # make a plot
        fig,ax = plt.subplots( figsize=(12,12) )
        ax.imshow( im, cmap='gray' )
        for coord in object_coords:
            boxhalfwidth = 5
            patchxcoord = coord[1]
            patchycoord = coord[0]
            anchor = (patchxcoord-boxhalfwidth,patchycoord-boxhalfwidth)
            box = mpl.patches.Rectangle(anchor, 2*boxhalfwidth, 2*boxhalfwidth,
                               edgecolor='r', facecolor=(0,0,0,0))
            ax.add_patch(box)
            
fig,ax = plt.subplots()
xax = np.arange(0,nmaxobjects+1)
ax.step(xax, ntrue_hist, color='b', label='True object count')
ax.step(xax,nreco_hist, color='r', label='Reconstructed object count')
ax.legend()